In [1]:
#libraries
import os
import pandas as pd
from datetime import datetime as dt
from datetime import time as t
from datetime import timedelta
from pandas.tseries.offsets import MonthEnd
from IPython.display import display
import sqlite3
import json
import numpy as np
import re
from openpyxl import load_workbook
import xlsxwriter
import requests
from requests.auth import HTTPBasicAuth

In [2]:
#FUNCTION TO IMPORT FOLDERS
def import_csv(path):
    raw = []
    for file in os.listdir(path):
        if file.endswith(".csv"):
            file_path = os.path.join(path, file)
            file_data = pd.read_csv(file_path)
            file_data["filename"] = file
            raw.append(file_data)
            final_raw = pd.concat(raw, ignore_index=True)
    return final_raw
    
def import_xlsx(path, sheet_name):
    raw = []
    for file in os.listdir(path):
        if file.endswith(".xlsx"):
            file_path = os.path.join(path, file)
            file_data = pd.read_excel(file_path, engine="openpyxl", sheet_name = sheet_name)
            file_data["filename"] = file
            raw.append(file_data)
            final_raw = pd.concat(raw, ignore_index=True)
    return final_raw

In [3]:
# source
personal_path = os.environ['USERPROFILE']
if personal_path in ['C:\\Users\\dinhhoang.nguyen.CONCENTRIX', 'C:\\Users\\ADMIN']:
    middle_path = r'OneDrive - Concentrix Corporation\WFM-internal'
else:
    middle_path = r'Concentrix Corporation\Dinh Hoang Nguyen - WFM-internal'
user_credential = os.path.join(os.environ['USERPROFILE'], middle_path)
# EPS path
eps_path = os.path.join("C:", user_credential, r"Data\EPS Tableau")

In [4]:
# IMPORT MASTER ROSTER
masterroster = pd.read_json(os.path.join(user_credential, r'DB\filejson\masterroster.json'))
masterroster_daily=masterroster[['Employee_ID','TED Name', 'PST_Start_Date', 'Role','Booking Login ID','Full name','Wave #']]
masterroster_daily['Employee_ID'] = masterroster_daily['Employee_ID'].astype("Int64")

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20908\242608621.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  masterroster_daily['Employee_ID'] = masterroster_daily['Employee_ID'].astype("Int64")


In [5]:
# IMPORT OT
ot = pd.read_json(os.path.join(user_credential, r'DB\filejson\ot_full.json'))
ot_daily=ot[['Emp ID','Date','OT']]
ot_daily=ot_daily.groupby(['Emp ID','Date'], as_index=False).sum()
ot_daily['Emp ID']=ot_daily['Emp ID'].astype("Int64")
ot_daily['Date'] = pd.to_datetime(ot_daily['Date'], format="mixed").dt.date

In [6]:
# IMPORT RAMCO
ramco_daily = pd.read_json(os.path.join(user_credential, r'DB\filejson\ramco_full.json'))
ramco_daily['Date']=pd.to_datetime(ramco_daily['Date'],format='mixed').dt.date
ramco_daily['EID'] = ramco_daily['EID'].astype("Int64")
ramco_daily['Value']=ramco_daily['Value'].fillna(0)
ramco_daily=ramco_daily.rename(columns={'Value':'RAMCO'})
ramco_ot_daily=pd.merge(ramco_daily,ot_daily,left_on=['EID','Date'],right_on=['Emp ID','Date'],how='left')
def OTREGISTERED_func(x):
    if x['RAMCO'] =='PO' or x['RAMCO'] =='PH' :
        return x['OT']/24
    elif x['RAMCO'] =='WO' or x['RAMCO'] =='AL' or x['RAMCO'] =='SL' or x['RAMCO'] =='HO':
        return 0 
    elif x['RAMCO'] =='HAL' :
        return 3.75/24
    else:
        return x['OT']/24+7.5/24
ramco_ot_daily['OT Registered'] = ramco_ot_daily.apply(OTREGISTERED_func, axis=1)
ramco_ot_daily=ramco_ot_daily[['EID','Date','RAMCO','OT','OT Registered']]

In [7]:
# IMPORT RAMCOOT
ramcoot = pd.read_json(os.path.join(user_credential, r'DB\filejson\ramcoot_full.json'))

In [8]:
# IMPORT SCHEDULE
schedule = pd.read_json(os.path.join(user_credential, r'DB\filejson\schedule_full.json'))
schedule['Date']=pd.to_datetime(schedule['Date']).dt.date

In [9]:
#IMPORT CUIC
cuic = pd.read_json(os.path.join(user_credential, r'DB\filejson\cuic_full.json'))

In [10]:
#IMPORT CPI
cpi = pd.read_json(os.path.join(user_credential, r'DB\filejson\cpi_full.json'))
cpi_daily=cpi[['Staff Name', 'Employee_ID', 'Date', 'Channel', 'Number of Records']]

channel_list = ['phone','email','messaging','Live Chat']
for i in channel_list:
    cpi_daily[i] = cpi_daily.apply(lambda x: x['Number of Records'] if x['Channel'] == i else 0, axis=1)
cpi_daily['Other channels Cases #'] = cpi_daily.apply(lambda x: x['Number of Records'] if x['Channel'] not in channel_list else 0, axis=1)

non_phone_channel_list=['email','message','Live Chat']
cpi_daily['Non-phone Cases #']= cpi_daily.apply(lambda x: x['Number of Records'] if x['Channel'] in non_phone_channel_list else 0, axis=1)

cpi_daily=cpi_daily.rename(columns={'Number of Records':'Cases #',
                                   'phone':'Phone Cases #',
                                   'email':'Email Cases #',
                                   'messaging':'Messaging Cases #',
                                   'Live Chat':'Live Chat Cases #'})
cpi_daily=cpi_daily.drop(columns=['Staff Name','Channel'])
cpi_daily['Date']=pd.to_datetime(cpi_daily['Date'],format='mixed').dt.date
cpi_daily=cpi_daily.groupby(['Employee_ID','Date'], as_index=False).sum()
cpi_daily['Employee_ID'] = cpi_daily['Employee_ID'].astype("Int64")

In [11]:
# IMPORT TL
tl = pd.read_json(os.path.join(user_credential, r'DB\filejson\tl.json'))

In [12]:
# IMPORT EXCEPTION
exception = pd.read_json(os.path.join(user_credential, r'DB\filejson\exceptional_hrs.json'))
exception['Date']=pd.to_datetime(exception['Date'],format='mixed').dt.date
exception_daily=exception[['Emp ID','Date','Exception request \n(Minute)']]
exception_daily['Exception request \n(Minute)'] = exception_daily['Exception request \n(Minute)'].astype("float")
exception_daily['Exception request \n(Minute)']=exception['Exception request \n(Minute)'].fillna(0)
exception_daily=exception_daily.rename(columns={'Exception request \n(Minute)':'Exception request Hours'})
exception_daily=exception_daily.groupby(['Emp ID','Date'], as_index=False).sum()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20908\2637013935.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exception_daily['Exception request \n(Minute)'] = exception_daily['Exception request \n(Minute)'].astype("float")
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20908\2637013935.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exception_daily['Exception request \n(Minute)']=exception['Exception request \n(Minute)'].fillna(0)


In [13]:
#IMPORT EPS
eps_raw = import_csv(path = eps_path)
eps = eps_raw.drop(columns=['Index', 'filename', 'Date'])
eps['Session Login'] = pd.to_datetime(eps['Session Login'], errors='coerce')
eps['Session Login'] = eps['Session Login'].dt.tz_localize('Europe/Berlin', ambiguous=True).dt.tz_convert('UTC')
eps['Session Logout'] = pd.to_datetime(eps['Session Logout'], errors='coerce')
eps['Session Logout'] = eps['Session Logout'].dt.tz_localize('Europe/Berlin', ambiguous=True).dt.tz_convert('UTC')
eps['Session Login'] =pd.to_datetime(eps['Session Login'], errors='coerce').dt.tz_localize(None)
eps['Session Logout'] =pd.to_datetime(eps['Session Logout'], errors='coerce').dt.tz_localize(None)

eps['Session login VN'] = pd.to_datetime(eps['Session Login'], format ='mixed') + pd.Timedelta(hours=7)
eps['Session logout VN'] = pd.to_datetime(eps['Session Logout'], format ='mixed') + pd.Timedelta(hours=7)
eps['Session login date'] = pd.to_datetime(eps['Session login VN'], format ='mixed').dt.date
eps['Session login time'] = pd.to_datetime(eps['Session login VN'], format ='mixed').dt.time
eps['Session login hour'] = pd.to_datetime(eps['Session login VN'], format ='mixed').dt.strftime('%H')
eps['Session logout date'] = pd.to_datetime(eps['Session logout VN'], format ='mixed').dt.date
eps['Session logout time'] = pd.to_datetime(eps['Session logout VN'], format ='mixed').dt.time
eps['Session logout hour'] = pd.to_datetime(eps['Session logout VN'], format ='mixed').dt.strftime('%H')
eps['Session login hour']=eps['Session login hour'].astype("Int64")
eps['Session logout hour']=eps['Session logout hour'].astype("Int64")
nighthour_23h="23:00:00"
nighthour_7h="07:00:00"
nighthour_24h="00:00:00"
eps['Session login 23h'] = pd.to_datetime(eps['Session login date'].astype(str) + nighthour_23h, format = '%Y-%m-%d%H:%M:%S', errors='coerce')
eps['Session logout 23h'] = pd.to_datetime(eps['Session logout date'].astype(str) + nighthour_23h, format = '%Y-%m-%d%H:%M:%S', errors='coerce')
eps['Session login 7h'] = pd.to_datetime(eps['Session login date'].astype(str) + nighthour_7h, format = '%Y-%m-%d%H:%M:%S', errors='coerce')
eps['Session logout 7h'] = pd.to_datetime(eps['Session logout date'].astype(str) + nighthour_7h, format = '%Y-%m-%d%H:%M:%S', errors='coerce')
eps['Session login 24h'] = pd.to_datetime(eps['Session login date'].astype(str) + nighthour_24h, format = '%Y-%m-%d%H:%M:%S', errors='coerce')+pd.Timedelta(days=1)
eps['Session logout 24h'] = pd.to_datetime(eps['Session logout date'].astype(str) + nighthour_24h, format = '%Y-%m-%d%H:%M:%S', errors='coerce')+pd.Timedelta(days=0)
eps=eps[eps['EID'].isnull() == False]
def night_func(x):
    if (x['Session login hour']>=23 and x['Session logout hour']>=23) or (x['Session login hour']>=23 and x['Session logout hour']<7) or (x['Session login hour']<7 and x['Session logout hour']<7):
        return x['Total Time']
    elif x['Session login hour']>=23 and x['Session logout hour']>=7:
        if ((x['Session login 24h']-x['Session login VN'])+pd.Timedelta(hours=7)).total_seconds()<x['Total Time']:
            return ((x['Session login 24h']-x['Session login VN'])+pd.Timedelta(hours=7)).total_seconds()
        else :
            return x['Total Time']
    elif x['Session login hour']<7 and x['Session logout hour']>=7 and x['Session logout hour']<23:
        if (x['Session login 7h']-x['Session login VN']).total_seconds()<x['Total Time']:
            return (x['Session login 7h']-x['Session login VN']).total_seconds()
        else :
            return x['Total Time']
    elif x['Session login hour']<7 and x['Session logout hour']>=23:
        if ((x['Session login 7h']-x['Session login VN'])+(x['Session logout VN']-x['Session logout 23h'])).total_seconds()<x['Total Time']:
            return ((x['Session login 7h']-x['Session login VN'])+(x['Session logout VN']-x['Session logout 23h'])).total_seconds()
        else :
            return x['Total Time']
    elif x['Session login hour']<23 and x['Session logout hour']>=23:
        if (x['Session logout VN']-x['Session logout 23h']).total_seconds()<x['Total Time']:
            return (x['Session logout VN']-x['Session logout 23h']).total_seconds()
        else :
            return x['Total Time']
    elif x['Session login hour']<23 and x['Session logout hour']<7:
        if (x['Session logout VN']-x['Session login 23h']).total_seconds()<x['Total Time']:
            return (x['Session logout VN']-x['Session login 23h']).total_seconds()
        else :
            return x['Total Time']
    else:
        return (x['Session login VN']-x['Session login VN']).total_seconds()
eps['night']= eps.apply(night_func, axis=1)

eps = eps.drop(columns=['Session login VN', 'Session logout VN', 'Username', 'manager_username', 'sitecode', 'Session Time',
                        'Session login hour','Session logout hour','Session login 23h','Session login 7h','Session login 24h'
                       ,'Session logout 23h','Session logout 7h','Session logout 24h'])
eps['EID']=eps['EID'].astype("Int64")
# 1. Merge EPS and Schedule on EID and Date to get Shift_type, Shift, Session Date and Duration and change Date into VN Date
eps_schedule=pd.merge(eps, schedule, left_on=['EID','Session login date'], right_on=['Emp ID','Date'], how='left')
eps_schedule=eps_schedule[['EID','Session Login','Session Logout','BPE Code','Total Time','Session login date','Session login time',
                           'Session logout date','Session logout time','LOB','Shift_type','Shift','night']]
eps_schedule['Date-1']=pd.to_datetime(eps_schedule['Session login date'], format ='mixed') - pd.Timedelta(1,"d")
eps_schedule['Date-1']=pd.to_datetime(eps_schedule['Date-1'], format ='mixed').dt.date
eps_schedule = pd.merge(eps_schedule, schedule, left_on=['EID','Date-1'], right_on =['Emp ID','Date'], how='left')
mytime=t(12,0,0)
condition=(eps_schedule['Shift_type_x']!='DS')&(eps_schedule['Shift_type_y']=='NS')&(eps_schedule['Session login time']<mytime)
eps_schedule=eps_schedule.rename(columns={'LOB_x': 'LOB','Shift_type_x':'Shift_type','Shift_x':'Shift'})
eps_schedule['Session Date'] = np.where(condition, eps_schedule['Date-1'], eps_schedule['Session login date'])
eps_schedule['Duration']=eps_schedule['Total Time']/3600
eps_schedule['night']=eps_schedule['night']/3600
eps_schedule=eps_schedule[['EID','Session Login','Session Logout','BPE Code','Total Time','Session login date','Session login time',
                           'Session logout date','Session logout time','LOB','Session Date','Duration','night']]
eps_schedule = pd.merge(eps_schedule, schedule, left_on=['EID','Session Date'], right_on =['Emp ID','Date'], how='left')
eps_schedule=eps_schedule.rename(columns={'LOB_y': 'LOB'})
eps_schedule=eps_schedule[['EID','Session Login','Session Logout','BPE Code','Total Time','Session login date','Session login time',
                           'Session logout date','Session logout time','LOB','Shift_type','Session Date','Duration','Shift','night']]

# 2. Add Week & Day Name and BPE Code Duration and then group by EID, Session Date
eps_dailytracking=eps_schedule
eps_dailytracking['Session Date'] = pd.to_datetime(eps_dailytracking['Session Date'],errors ='coerce')
# Add Week & Day Name column
eps_dailytracking['Week'] = eps_dailytracking['Session Date'].dt.strftime('%Y%W')
eps_dailytracking['Day Name'] = eps_dailytracking['Session Date'].dt.day_name()

# Add Picklist,Ready or Talking,..BPE Code
bpe_code=['Picklist - off Phone','Ready or Talking','Finalize Call','Meeting','Training','New Hire Training','Lunch',
          'Break','RONA','Unscheduled Picklist','Payment Processing','Social Media','Mass Issue','Project']
for i in bpe_code:
    eps_dailytracking[i] = eps_dailytracking.apply(lambda x: x['Duration'] if x['BPE Code'] == i else 0, axis=1)
eps_dailytracking['Other codes'] = eps_dailytracking.apply(lambda x: x['Duration'] if x['BPE Code'] not in bpe_code else 0, axis=1)

night_bpe_code=['Picklist - off Phone','Ready or Talking','Finalize Call','Meeting','Training']
for i in night_bpe_code:
    eps_dailytracking['Night '+i] = eps_dailytracking.apply(lambda x: x['night'] if x['BPE Code'] == i else 0, axis=1)

eps_dailytracking['Night Productive Hours']=eps_dailytracking['Night Picklist - off Phone']+eps_dailytracking['Night Ready or Talking']+eps_dailytracking['Night Finalize Call']
eps_dailytracking['Night Downtime Hours']=eps_dailytracking['Night Training']+eps_dailytracking['Night Meeting']
eps_dailytracking=eps_dailytracking.drop(columns=['Session Login','Session Logout','BPE Code','Total Time','Session login date',
                                                  'Session login time','Session logout date','Session logout time','night'])
eps_dailytracking=eps_dailytracking.groupby(['EID','LOB','Shift_type','Session Date','Shift','Week','Day Name'], as_index=False).sum()
eps_dailytracking['Session Date']=pd.to_datetime(eps_dailytracking['Session Date'],format='mixed').dt.date


In [14]:
# Login_Logout Time
login_logout=eps_schedule[['EID','Session Date','Shift_type','Session login time','Session logout time']]

# Login_Logout Time: filter !=NS
login_logout_ds=login_logout.loc[(login_logout['Shift_type'] !='NS')]
login_logout_ds=login_logout_ds.groupby(['EID','Session Date','Shift_type'], as_index=False).agg(min=('Session login time', 'min'), max=('Session logout time', 'max'))
login_logout_ds=login_logout_ds.rename(columns={'min':'Login Time','max':'Logout Time'})

# Login_Logout Time: filter NS (condition: login sau 15h & logout >15h)
validate_time=t(15,0,0)
login_ns=login_logout.loc[(login_logout['Shift_type'] =='NS')&(login_logout['Session login time'] > validate_time)]
login_ns=login_ns.groupby(['EID','Session Date','Shift_type'], as_index=False).agg(min=('Session login time', 'min'))
logout_ns=login_logout.loc[(login_logout['Shift_type'] =='NS')&(login_logout['Session logout time'] < validate_time)]
logout_ns=logout_ns.groupby(['EID','Session Date','Shift_type'], as_index=False).agg(max=('Session logout time', 'max'))
login_logout_ns=pd.merge(login_ns,logout_ns,left_on=['EID','Session Date','Shift_type'],right_on=['EID','Session Date','Shift_type'],how='left')
login_logout_ns=login_logout_ns.rename(columns={'min':'Login Time','max':'Logout Time'})

# Login_Logout Time:Combine DS & NS login logout time
login_logout=pd.concat([login_logout_ds, login_logout_ns], ignore_index=True, sort=False)

In [15]:
### Daily tracking
# Merge CPI to get Case #
dailytracking=pd.merge(eps_dailytracking,cpi_daily,left_on=['EID','Session Date'],right_on=['Employee_ID','Date'],how='left')
dailytracking=dailytracking.drop(columns=['Employee_ID','Date'])

# Merge Exception to get Exception minute
dailytracking=pd.merge(dailytracking,exception_daily,left_on=['EID','Session Date'],right_on=['Emp ID','Date'],how='left')
dailytracking=dailytracking.drop(columns=['Emp ID','Date'])

# Merge Ramco & OT to get Ramco, OT, OT Registered
dailytracking=pd.merge(dailytracking,ramco_ot_daily,left_on=['EID','Session Date'],right_on=['EID','Date'],how='left')
dailytracking=dailytracking.drop(columns=['Date'])

# Merge Master Roster to get Full name, Booking Login ID, TED Name, Wave #, Employee_ID
dailytracking=pd.merge(dailytracking,masterroster_daily,left_on=['EID'],right_on=['Employee_ID'],how='left')
dailytracking=dailytracking.drop(columns=['Employee_ID'])

# Merge TL to get Supervisor
dailytracking=pd.merge(dailytracking,tl,left_on=['EID'],right_on=['Emp ID'],how='left')
dailytracking=dailytracking.drop(columns=['Emp ID'])

# Merge Login Logout Time
login_logout['Session Date']=pd.to_datetime(login_logout['Session Date']).dt.date
dailytracking=pd.merge(dailytracking,login_logout,left_on=['EID','Session Date','Shift_type'],right_on=['EID','Session Date','Shift_type'],how='left')

dailytracking['Cases #']=dailytracking['Cases #'].fillna(0)
dailytracking['Phone Cases #']=dailytracking['Phone Cases #'].fillna(0)
dailytracking['Non-phone Cases #']=dailytracking['Non-phone Cases #'].fillna(0)
dailytracking['RAMCO']=dailytracking['RAMCO'].fillna(0)
dailytracking['OT']=dailytracking['OT'].fillna(0)
dailytracking['Exception request Hours']=dailytracking['Exception request Hours']/60
dailytracking['Exception request Hours']=dailytracking['Exception request Hours'].fillna(0)

#Productive Hours including: Picklist - off Phone + Ready or Talking + Finalize Call
dailytracking['Productive hours']=dailytracking['Picklist - off Phone']+dailytracking['Ready or Talking']+dailytracking['Finalize Call']
#Downtime Hours including: Meeting + Training
dailytracking['Downtime Hours']=dailytracking['Meeting']+dailytracking['Training']
#Delivery Hours including: Productive Hours + Downtime Hours + New Hire Training
dailytracking['Delivery hours']=dailytracking['Productive hours']+dailytracking['Downtime Hours']+dailytracking['New Hire Training']
dailytracking=dailytracking.rename(columns={'Duration':'Staffed Hours'})
#CPH = sum(Cases #)/sum(Ready or Talking','Picklist - off Phone','Finalize Call',
#RONA,'Unscheduled Picklist','Payment Processing','Social Media','Mass Issue','Project')
dailytracking['CPH - Agent']=dailytracking['Cases #']/(dailytracking['Ready or Talking']+dailytracking['Picklist - off Phone']+
                                                        dailytracking['Finalize Call']+dailytracking['RONA']+
                                                        dailytracking['Unscheduled Picklist']+dailytracking['Payment Processing']+
                                                        dailytracking['Social Media']+ dailytracking['Mass Issue']+
                                                        dailytracking['Project'])
def standardtime_func(x):
    if x['RAMCO'] =='PR' or x['RAMCO'] =='PH' :
        return x['OT']+8
    elif x['RAMCO'] =='PO':
        return x['OT']
    else:
        return 0
dailytracking['Standard time']= dailytracking.apply(standardtime_func, axis=1)
dailytracking['Extra rendered hours']=(dailytracking['Delivery hours']+dailytracking['Break']+(dailytracking['Exception request Hours']/(60*24)))-dailytracking['Standard time']
def approvedovertime_func(x):
    if x['OT']<=0:
        return 0
    elif x['RAMCO'] not in ['PR','PO','PH']:
        return 0
    elif x['Extra rendered hours'] >= 0:
        return  x['OT']
    elif (x['OT']+x['Extra rendered hours']-0.25) < 0 :
        return  0
    else:         
        return x['OT']+x['Extra rendered hours']-0.25
dailytracking['Approved Overtime']= dailytracking.apply(approvedovertime_func, axis=1)
dailytracking['Approved Overtime']=dailytracking['Approved Overtime'].round(0)
dailytracking['Day Productive Hours']=dailytracking['Productive hours']-dailytracking['Night Productive Hours']
dailytracking['Day Downtime Hours']=dailytracking['Downtime Hours']-dailytracking['Night Downtime Hours']
dailytracking['Session Date'] = pd.to_datetime(dailytracking['Session Date'])
dailytracking=dailytracking.sort_values(by='Session Date',ascending=True)
#ATD MM
def atdmm_shift_func(x):
    if x['Shift'] in ['0400-1300','0500-1400','0600-1500','0700-1600','0800-1700','0900-1800','1000-1900','1100-2000',
                      '1200-2100','1300-2200','1400-2300','1500-2400','1600-0100','1700-0200',
                      '1800-0300','1900-0400','2000-0500','2100-0600','2200-0700','HAL','Training','PEGA']:
        return 'WORK'
    else:         
        return 'OFF'
dailytracking['ATD MM Shift']= dailytracking.apply(atdmm_shift_func, axis=1)
def atdmm_ramco_func(x):
    if x['RAMCO'] in ['PH','PO','PR','PI','POWH','HAL','HLWP','HSL']:
        return 'WORK'
    else:         
        return 'OFF'
dailytracking['ATD MM Ramco']= dailytracking.apply(atdmm_ramco_func, axis=1)
def atdmm_func(x):
    if x['EID'] =='':
        return ''
    elif x['RAMCO']=='PO' and x['Shift']=='OFF':
        return 'Valid'   
    elif x['RAMCO']=='' and x['Shift']!='':
        return '' 
    elif x['RAMCO']!='' and x['Shift']=='':
        return ''     
    elif x['RAMCO']=='' and x['Shift']=='':
        return 'Valid'  
    elif x['RAMCO']=='AB' and x['ATD MM Shift']=='WORK':
        return 'Valid' 
    elif x['ATD MM Ramco'] == x['ATD MM Shift']:
        return 'Valid' 
    else:         
        return 'ATD MM'
dailytracking['ATD MM']= dailytracking.apply(atdmm_func, axis=1)
#PO (MTD)
def po_func(x):
    if x['RAMCO'] in ['PO']:
        return 1
    else:         
        return 0
dailytracking['PO']= dailytracking.apply(po_func, axis=1)
dailytracking['Session Date'] = pd.to_datetime(dailytracking['Session Date'])
dailytracking['#PO (MTD)'] = dailytracking.groupby([dailytracking['Session Date'].dt.to_period('m'),'EID'])['PO'].cumsum()
#POHrs (MTD)
def pohour_func(x):
    if x['RAMCO'] in ['PO']:
        return x['Approved Overtime']
    else:         
        return 0
dailytracking['PO Hour']= dailytracking.apply(pohour_func, axis=1)
dailytracking['POHrs(MTD)'] = dailytracking.groupby([dailytracking['Session Date'].dt.to_period('m'),'EID'])['PO Hour'].cumsum()
#OT (MTD)
dailytracking['OT (MTD)'] = dailytracking.groupby([dailytracking['Session Date'].dt.to_period('m'),'EID'])['Approved Overtime'].cumsum()
# Final OT
def finalOT_func(x):
    if x['OT (MTD)'] <= 40 :
        return x['Approved Overtime']
    else:         
        return 0
dailytracking['Final OT']= dailytracking.apply(finalOT_func, axis=1)
#OT OTP
def OT_OTP_func(x):
    if x['OT (MTD)'] > 40 :
        return x['Approved Overtime']
    else:         
        return 0
dailytracking['OT OTP']= dailytracking.apply(OT_OTP_func, axis=1)
#OT-LessDelivery
def OT_LessDelivery_func(x):
    if x['Approved Overtime'] > x['OT'] :
        return 1
    else:         
        return 0
dailytracking['OT-LessDelivery']= dailytracking.apply(OT_LessDelivery_func, axis=1)
#OT>4Hrs On PR & OT>8Hrs On PO
def OT_on_PO_func(x):
    if x['RAMCO'] =='PR' and x['Approved Overtime']>4 :
        return 1
    elif x['RAMCO'] =='PO' and x['Approved Overtime']>8 :
        return 1 
    else:         
        return 0
dailytracking['OT>4Hrs On PR \n OT>8Hrs On PO']= dailytracking.apply(OT_on_PO_func, axis=1)
#PR<8.75
def PR_func(x):
    if (x['Staffed Hours'] + x['Exception request Hours'])<8.75 :
        return 1
    else:         
        return 0
dailytracking['PR < 8.75']= dailytracking.apply(PR_func, axis=1)
#PR (MTD)
def PRMTD_func(x):
    if x['EID']=='' :
        return 0
    else:         
        return 1
dailytracking['PR count']= dailytracking.apply(PRMTD_func, axis=1)
dailytracking['PR (MTD)'] = dailytracking.groupby([dailytracking['Session Date'].dt.to_period('m'),'EID'])['PR count'].cumsum()
#Low PerF
def LowPerF_func(x):
    if x['CPH - Agent']<2 :
        return 1
    else:         
        return 0
dailytracking['Low PerF']= dailytracking.apply(LowPerF_func, axis=1)
#Login On Dayoff
def LoginonDayoff_func(x):
    if x['ATD MM Shift'] =='OFF' and x['ATD MM Ramco']=='OFF' and x['Staffed Hours']>0:
        return 1
    else:         
        return 0
dailytracking['Login on Dayoff']= dailytracking.apply(LoginonDayoff_func, axis=1)

#Not Logout
import re
def split1_it(startshift):
    return re.findall('(\d{4})-', startshift)
dailytracking['Shift1'] = dailytracking['Shift'].apply(split1_it)
dailytracking['Shift1']=dailytracking['Shift1'].astype(str)
dailytracking['Shift1'] = dailytracking['Shift1'] .str.replace("['", "")
dailytracking['Shift1'] =dailytracking['Shift1'] .str.replace("']", "")
dailytracking['Shift1'] =dailytracking['Shift1'] .str.replace("00", "")
dailytracking['Shift1'] =dailytracking['Shift1'] .str.replace("[]", '')
def split_it(endshift):
    return re.findall('\d{4}-(\d{4})', endshift)
dailytracking['Shift2'] = dailytracking['Shift'].apply(split_it)
dailytracking['Shift2']=dailytracking['Shift2'].astype(str)
dailytracking['Shift2'] = dailytracking['Shift2'] .str.replace("['", "")
dailytracking['Shift2'] =dailytracking['Shift2'] .str.replace("']", "")
dailytracking['Shift2'] =dailytracking['Shift2'] .str.replace("00", "")
dailytracking['Shift2'] =dailytracking['Shift2'] .str.replace("[]", '')
def shift1_func(x):
    if x['Shift1'] =='':
        return 0
    else:         
        return x['Shift1']
dailytracking['Shift1']= dailytracking.apply(shift1_func, axis=1)
dailytracking['Shift1']=dailytracking['Shift1'].astype('Float64')
dailytracking['Shift1']=dailytracking['Shift1'].astype('Int64')
def shift2_func(x):
    if x['Shift2'] =='':
        return 0
    else:         
        return x['Shift2']
dailytracking['Shift2']= dailytracking.apply(shift2_func, axis=1)
dailytracking['Shift2']=dailytracking['Shift2'].astype('Float64')
dailytracking['Shift2']=dailytracking['Shift2'].astype('Int64')
def time1_func(x):
    if x['Shift1'] =='':
        return t(0,0,0)
    elif x['Shift1'] ==24:
        return t(23,59,59)
    else:         
        return t(x['Shift1'],0,0)
dailytracking['Time1']= dailytracking.apply(time1_func, axis=1)
def time_func(x):
    if x['Shift2'] =='':
        return t(0,0,0)
    elif x['Shift2'] ==24:
        return t(23,59,59)
    else:         
        return t(x['Shift2'],0,0)
dailytracking['Time2']= dailytracking.apply(time_func, axis=1)
dailytracking['Time1'] = pd.to_datetime(dailytracking['Time1'], format='%H:%M:%S', errors='coerce').dt.time
dailytracking['Time2'] = pd.to_datetime(dailytracking['Time2'], format='%H:%M:%S', errors='coerce').dt.time
def nightshift_func(x):
    if x['Shift_type'] =="OFF":
        return x['Session Date']
    elif x['Shift2'] < 12:
        return x['Session Date']+ pd.Timedelta(days=1)
    else:         
        return x['Session Date']
dailytracking['Date']= dailytracking.apply(nightshift_func, axis=1) 
dailytracking['datetime1'] = pd.to_datetime(dailytracking['Session Date'].astype(str) + dailytracking['Time1'].astype(str), format = '%Y-%m-%d%H:%M:%S')
dailytracking['datetime2'] = pd.to_datetime(dailytracking['Date'].astype(str) + dailytracking['Time2'].astype(str), format = '%Y-%m-%d%H:%M:%S')
dailytracking['Approved Overtime1']=dailytracking['Approved Overtime'].astype('Float64')
dailytracking['Approved Overtime1']=dailytracking['Approved Overtime1'].apply(np.ceil)
dailytracking['datetime2'] = pd.to_datetime(dailytracking['datetime2'])
dailytracking['Approved Overtime1'] = pd.to_timedelta(dailytracking['Approved Overtime1'],'h')
dailytracking['datetime2'] = dailytracking['datetime2']+dailytracking['Approved Overtime1']
dailytracking['datetime2'] = dailytracking['datetime2']+ pd.Timedelta(minutes=5)
dailytracking['datetime1'] = pd.to_datetime(dailytracking['datetime1'])+ pd.Timedelta(minutes=5)
dailytracking['Logout Time']=dailytracking['Logout Time'].fillna(0)
dailytracking['datetime_logout'] = pd.to_datetime(dailytracking['Date'].astype(str) + dailytracking['Logout Time'].astype(str), format = '%Y-%m-%d%H:%M:%S', errors='coerce')
dailytracking['Login Time']=dailytracking['Login Time'].fillna(0)
dailytracking['datetime_login'] = pd.to_datetime(dailytracking['Session Date'].astype(str) + dailytracking['Login Time'].astype(str), format = '%Y-%m-%d%H:%M:%S', errors='coerce')
def notlogout_func(x):
    if x['Shift'] in {'AL','OFF','Training','HAL','CO','UPL'}:
        return ""
    elif x['datetime_logout'] > x['datetime2']:
        return "not logout after shift end"       
    else:         
        return ""
dailytracking['Not Logout']= dailytracking.apply(notlogout_func, axis=1)

# Late-Soon Login Logout
def late_func(x):
    if x['Shift'] in {'AL','OFF','Training','HAL','CO','UPL'}:
        return ""
    elif x['datetime_login'] > x['datetime1']:
        return "Late"       
    else:         
        return ""
dailytracking['Late']= dailytracking.apply(late_func, axis=1)
def soon_func(x):
    if x['Shift'] in {'AL','OFF','Training','HAL','CO','UPL'}:
        return ""
    elif x['datetime_logout'] < x['datetime2']- pd.Timedelta(minutes=5):
        return "Soon"       
    else:         
        return ""
dailytracking['Soon']= dailytracking.apply(soon_func, axis=1)
def latesoon_func(x):
    if x['Late'] =="" and x['Soon']=="":
        return ""
    elif x['Late']!="" and x['Soon']=="":
        return x['Late']       
    elif x['Late']=="" and x['Soon']!="":
        return x['Soon']   
    else:         
        return x['Late']+'-'+x['Soon']
dailytracking['Late-Soon']= dailytracking.apply(latesoon_func, axis=1)

# Overconsecutive
dailytracking= dailytracking.sort_values(by = ['EID', 'Session Date'])
dailytracking['level_1']=(dailytracking['Session Date'] - dailytracking['Session Date'].shift(1)).astype('int64')/86400000000000 #to get the discrepancy between current and previous visible working day
dailytracking['pre_ramco']=dailytracking['RAMCO'].shift(1)
def level1_func(x):
    if x['RAMCO'] in {'MTLX', 'MTLA', 'LWP', 'AL', 'AB', 'WO', 'SL', 'NM', 'MTL', 'CSLA', 'MOE', 'NCH', 'CSLB', 'PTL', 'PTL', 'PNL', 'CAL', 'HO', 'CDH', 'CO', 'CM', 'GM', 'MSL', 'VDH', 'MOC', 'CMLF'}:
        return 2
    elif x['level_1']==1 and x['pre_ramco'] in {'MTLX', 'MTLA', 'LWP', 'AL', 'AB', 'WO', 'SL', 'NM', 'MTL', 'CSLA', 'MOE', 'NCH', 'CSLB', 'PTL', 'PTL', 'PNL', 'CAL', 'HO', 'CDH', 'CO', 'CM', 'GM', 'MSL', 'VDH', 'MOC', 'CMLF'}:
        return 2
    else:         
        return x['level_1']
dailytracking['level_1']= dailytracking.apply(level1_func, axis=1)

dailytracking['check_eid'] = (dailytracking['EID'] - dailytracking['EID'].shift(1))

dailytracking[ 'level_2'] = ((dailytracking['level_1']!=1) | (dailytracking['check_eid'].isna()) | (dailytracking['check_eid']!=0)).cumsum()

dailytracking['OverConsecutive'] = (
    dailytracking[['level_2']]
    .assign(OverConsecutive=1)
    .groupby('level_2')['OverConsecutive']
    .transform('cumsum')
)
dailytracking['OverConsecutive'] = dailytracking.apply(lambda x: 'OverConsecutive' if x['OverConsecutive'] > 6 else x['OverConsecutive'], axis=1)
dailytracking['Staffed Hours']=dailytracking['Staffed Hours']

limit_date = max(pd.to_datetime(dailytracking['Session Date'], format ='mixed')) - pd.Timedelta(days=1)
dailytracking=dailytracking.loc[dailytracking['Session Date'] <= limit_date]

dailytracking['Session Date'] = pd.to_datetime(dailytracking['Session Date'], format='mixed').dt.date
dailytracking_raw = dailytracking.drop(columns=['ATD MM Shift','ATD MM Ramco','PO','PO Hour','PR count','level_1','pre_ramco','check_eid',
                                            'level_2','Shift1','Shift2','Time1','Time2','Date','datetime1','datetime2','Approved Overtime1',
                                            'datetime_logout','datetime_login','Late','Soon','CPH - Agent'])
dailytracking_raw['Session Date'] = pd.to_datetime(dailytracking_raw['Session Date'], format='mixed').dt.date
dailytracking_raw.replace([np.inf, -np.inf], 0, inplace=True)
dailytracking_raw=dailytracking_raw.drop(columns=['OT','PST_Start_Date','Role','Standard time','Extra rendered hours'])

In [16]:
# ADHERENCE
workplan = pd.read_json(os.path.join(r'C:', user_credential, r'DB\filejson\workplan_merge_iex.json'))
workplan=workplan[['Date','Agent ID','Scheduled Activity','Length','EID']]
workplan['Date']=pd.to_datetime(workplan['Date']).dt.date
workplan['EID']=workplan['EID'].astype("Int64")

workplan['Plotted Downtime Hours']=workplan.apply(lambda x: x['Length'] if x['Scheduled Activity'] in {'Coaching 1:1','Team Meeting'} else 0, axis=1)
workplan['Plotted Phone Hours']=workplan.apply(lambda x: x['Length'] if x['Scheduled Activity']=='Open Time' else 0, axis=1)
workplan['Plotted Picklist Hours']=workplan.apply(lambda x: x['Length'] if x['Scheduled Activity'] =='Email 1' else 0, axis=1)
workplan['Plotted Lunch Hours']=workplan.apply(lambda x: x['Length'] if x['Scheduled Activity'] =='Lunch' else 0, axis=1)
workplan['Plotted Break Hours']=workplan.apply(lambda x: x['Length'] if x['Scheduled Activity'] =='Break Offline' else 0, axis=1)

workplan['Plotted Downtime Hours']=workplan['Plotted Downtime Hours']*24
workplan['Plotted Phone Hours']=workplan['Plotted Phone Hours']*24
workplan['Plotted Picklist Hours']=workplan['Plotted Picklist Hours']*24
workplan['Plotted Lunch Hours']=workplan['Plotted Lunch Hours']*24
workplan['Plotted Break Hours']=workplan['Plotted Break Hours']*24
workplan=workplan.groupby(['Date','Agent ID','EID'], as_index=False).sum()
workplan['Plotted Producted Hour']=workplan['Plotted Phone Hours']+workplan['Plotted Picklist Hours']

eps_adherence=dailytracking[['Session Date','EID','Full name','Booking Login ID','LOB','Supervisor','Wave #',
                                 'Picklist - off Phone','Ready or Talking','Meeting','Training','Lunch','Break']]
eps_adherence['Actual Downtime Hours']=eps_adherence['Meeting']+eps_adherence['Training']
eps_adherence['Session Date']=pd.to_datetime(eps_adherence['Session Date']).dt.date
adherence=pd.merge(eps_adherence,workplan,left_on=['Session Date','EID'],right_on=['Date','EID'],how='left')
adherence['Actual Producted Hour']=adherence['Picklist - off Phone']+adherence['Ready or Talking']
adherence=adherence.rename(columns={'Agent ID':'IEX ID','Picklist - off Phone':'Actual Picklist Hours','Ready or Talking':'Actual Phone Hours',
                                    'Lunch':'Actual Lunch Hours','Break':'Actual Break Hours'})
adherence=adherence[['Session Date','EID','Full name','IEX ID','Booking Login ID','LOB','Supervisor','Wave #',
                     'Plotted Downtime Hours','Actual Downtime Hours','Plotted Producted Hour','Actual Producted Hour',
                     'Plotted Phone Hours','Actual Phone Hours','Plotted Picklist Hours','Actual Picklist Hours',
                     'Plotted Lunch Hours','Actual Lunch Hours','Plotted Break Hours','Actual Break Hours']]
def downtimedeliveryper(x):
    if x['Plotted Downtime Hours']==0 and x['Actual Downtime Hours']!=0:
        return 0
    elif x['Plotted Downtime Hours']==0 and x['Actual Downtime Hours']==0:
        return 1
    else:         
        return x['Actual Downtime Hours']/x['Plotted Downtime Hours']
adherence['%Downtime Delivery']= adherence.apply(downtimedeliveryper, axis=1)

def producteddeliveryper(x):
    if x['Plotted Producted Hour']==0 and x['Actual Producted Hour']!=0:
        return 0
    elif x['Plotted Producted Hour']==0 and x['Actual Producted Hour']==0:
        return 1
    else:         
        return x['Actual Producted Hour']/x['Plotted Producted Hour']
adherence['%Producted Delivery']= adherence.apply(producteddeliveryper, axis=1)
adherence['Lunch variance']=(adherence['Actual Lunch Hours']-adherence['Plotted Lunch Hours'])/adherence['Plotted Lunch Hours']
adherence['Break variance']=(adherence['Actual Break Hours']-adherence['Plotted Break Hours'])/adherence['Plotted Break Hours']
adherence=adherence[['Session Date','EID','IEX ID',
                     'Plotted Downtime Hours','Actual Downtime Hours','%Downtime Delivery','Plotted Producted Hour','Actual Producted Hour',
                     '%Producted Delivery','Plotted Phone Hours','Actual Phone Hours','Plotted Picklist Hours','Actual Picklist Hours',
                     'Plotted Lunch Hours','Actual Lunch Hours','Lunch variance','Plotted Break Hours','Actual Break Hours','Break variance']]
adherence.replace([np.inf, -np.inf], 0, inplace=True)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20908\330151357.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eps_adherence['Actual Downtime Hours']=eps_adherence['Meeting']+eps_adherence['Training']
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_20908\330151357.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eps_adherence['Session Date']=pd.to_datetime(eps_adherence['Session Date']).dt.date


In [17]:
daily_adherence=pd.merge(dailytracking_raw,adherence,left_on=['Session Date','EID'],right_on=['Session Date','EID'],how='left')
daily_adherence.to_csv(os.path.join(personal_path,r'Concentrix Corporation\Tung Quan Le - BKN\DailyTracking\[BcomDB] Dailytracking.csv'), index=False)